# Accculuaration gap
How different (in terms of cross-entrop) are posts from users within and outside a community?
Based on no country and Justine's later paper, right now we will look at, within a certian month, 

Follow this: 
    
- randomly sampling 200 active users—defined as users with at least 5 comments in the respective community and month.

- For each of these 200 active users we select 5 random 10-word spans from 5 unique comments.

- To ensure robustness and maximize data efficiency, we construct 100 SLMs for each community-month pair that
has enough data, bootstrap-resampling (resampling with replacement) from the set of active users.

In [1]:
import numpy as np
import csv as csv
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import math
import json
from datetime import datetime
from termcolor import colored
import os
from sklearn.model_selection import train_test_split
from langdetect import detect

from nltk.util import bigrams, ngrams
from nltk.lm.preprocessing import pad_both_ends, flatten, padded_everygram_pipeline
from nltk.lm.smoothing import GoodTuring
from nltk.tokenize import word_tokenize
from nltk.lm.models import InterpolatedLanguageModel, WittenBellInterpolated, KneserNeyInterpolated, MLE
from nltk.lm.api import LanguageModel
from nltk.lm import NgramCounter

os.chdir('../data/test_subs_2017/')


In [43]:
df_comments_science = pd.read_csv('../cleaned/science_comments_2017.csv')
df_posts_science = pd.read_csv('../cleaned/science_posts_2017.csv')
df_science_author_counts = pd.read_csv('../cleaned/author_count_science_2017.csv')

In [ ]:
##################################################################
# randomly sampling 200 active users  
# defined as users with at least 5 COMMENTS ONLY in the respective community and month.
##################################################################
def get_active_users(author_df, month, author_col, comments_only=True):
    if comments_only:
        df_author_comments = author_df[author_df['kind'] == 'comment']
        return df_author_comments[df_author_comments[month] > 5].sample(200)[author_col]
    else:
        return author_df[author_df[month] > 5].drop_duplicates().sample(200)[author_col]
    
##################################################################   
# For each of these 200 active users
# select 5 random 10-word spans from 5 unique comments
# ASSUMING: from that month
##################################################################
def get_random_span(text, length):
    text = word_tokenize(text)
    print(len(text))
    try: 
        beg = random.randint(0, len(text) - length)
        end = beg + 10
        return t[beg:end]
    except:
        raise IndexError("Error: index out of range, probably happened if you didn't clean comments to be at least 10 words long")

def get_active_user_posts(df, authors, month, month_col='created_month'):
    df_month = df[df[month_col] == month]
    df_month_author = df_month[df_month['author'].apply(lambda x: x in authors)]
    df_grouped = df_month_author.groupby('author')
    sampled_comments = []
    for a, g in df_grouped:
        sample = g.sample(5)['body'].apply(lambda x: word_tokenize(x)[:10]) #TODO - change to random span
        sampled_comments.extend(sample)
    return sampled_comments

##################################################################
# To ensure robustness and maximize data efficiency, 
# we construct 100 SLMs for each community-month pair that has enough data
##################################################################
def construct_LM(text, vocab=None):
    if vocab:
        train, _ =  padded_everygram_pipeline(2, text)
    else: 
        train, vocab =  padded_everygram_pipeline(2, text)
    lm = KneserNeyInterpolated(2)
    lm.fit(train, vocab)
    return lm

# returns dict of month:SLM
def build_monthly_SLM(df, author_counts, slm_count):
    months = list(range(1,13))
    slm_dict = {}
    for m in months:
        slms = []
        for i in range(1, slm_count + 1):
            active_users = get_active_users(author_counts, str(m), 'author')
            active_user_comments = get_active_user_posts(df, list(active_users), m)
            slm = construct_LM(active_user_comments)
            slms.append(slm)
        slm_dict[m] = slms
    return slm_dict

# get active authors for science in the first month
slms = build_monthly_SLM(df_comments_science, df_science_author_counts, 50)

In [ ]:
slms

In [69]:

        
sample_posts_active_jan = get_active_user_posts(df_comments_science, list(active_users_jan), 1.0)

In [80]:

    
t_lm = construct_SLM(sample_posts_active_jan)

In [88]:
print(t_lm.counts)

<NgramCounter with 2 ngram orders and 22144 ngrams>


In [66]:
df_comments_cleaned = pd.read_csv('../cleaned/cleaned_no_mod_comments_2017.csv')


/homes/gws/taugust/miniconda3/envs/reddit/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,14,15,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
### Making vocab
from nltk.lm import Vocabulary

vocab = Vocabulary(words, unk_cutoff=2)

In [68]:
df_comment_words = df_comments_cleaned.body.fillna(' ').apply(word_tokenize)

KeyboardInterrupt: 